In [1]:
import mxnet as mx

In [2]:
mnist = mx.test_utils.get_mnist()

In [3]:
mx.random.seed(42)

In [4]:
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()

In [5]:
batch_size = 100

In [6]:
train_iter = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size, shuffle=True)

In [7]:
val_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)

In [8]:
data = mx.sym.var('data')

In [9]:
data = mx.sym.flatten(data=data)

In [10]:
fc1 = mx.sym.FullyConnected(data=data, num_hidden=128)

In [12]:
act1 = mx.sym.Activation(data=fc1, act_type='relu')

In [14]:
fc2 = mx.sym.FullyConnected(data=act1, num_hidden=64)

In [15]:
act2 = mx.sym.Activation(data=fc2, act_type='relu')

In [17]:
fc3 = mx.sym.FullyConnected(data=act2, num_hidden=10)

In [18]:
mlp = mx.sym.SoftmaxOutput(data=fc3, name='softmax')

In [19]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

In [20]:
mlp_model = mx.mod.Module(symbol=mlp, context=ctx)

In [22]:
mlp_model.fit(train_iter,
             eval_data=val_iter,
             optimizer='sgd',
             optimizer_params={'learning_rate':0.1},
             eval_metric='acc',
             batch_end_callback = mx.callback.Speedometer(batch_size, 100),
             num_epoch=10)

INFO:root:Epoch[0] Batch [100]	Speed: 43832.53 samples/sec	accuracy=0.104257
INFO:root:Epoch[0] Batch [200]	Speed: 45828.27 samples/sec	accuracy=0.112500
INFO:root:Epoch[0] Batch [300]	Speed: 41473.97 samples/sec	accuracy=0.111800
INFO:root:Epoch[0] Batch [400]	Speed: 41487.92 samples/sec	accuracy=0.110400
INFO:root:Epoch[0] Batch [500]	Speed: 42576.36 samples/sec	accuracy=0.120500
INFO:root:Epoch[0] Train-accuracy=0.224242
INFO:root:Epoch[0] Time cost=1.417
INFO:root:Epoch[0] Validation-accuracy=0.284400
INFO:root:Epoch[1] Batch [100]	Speed: 62589.59 samples/sec	accuracy=0.447525
INFO:root:Epoch[1] Batch [200]	Speed: 61044.91 samples/sec	accuracy=0.696500
INFO:root:Epoch[1] Batch [300]	Speed: 59393.67 samples/sec	accuracy=0.772000
INFO:root:Epoch[1] Batch [400]	Speed: 58757.41 samples/sec	accuracy=0.808600
INFO:root:Epoch[1] Batch [500]	Speed: 56983.03 samples/sec	accuracy=0.823900
INFO:root:Epoch[1] Train-accuracy=0.838788
INFO:root:Epoch[1] Time cost=1.014
INFO:root:Epoch[1] Validat

In [24]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)

In [25]:
prob = mlp_model.predict(test_iter)

In [26]:
assert prob.shape == (10000, 10)

In [27]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)

In [28]:
acc = mx.metric.Accuracy()

In [29]:
mlp_model.score(test_iter, acc)

[('accuracy', 0.9703)]

In [30]:
print(acc)

EvalMetric: {'accuracy': 0.9703}


In [31]:
assert acc.get()[1] > 0.96, 'Achieved accuracy (%f) is lower than expected (0.96)' %acc.get()[1]